In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import os
# os.chdir("../")


In [2]:
data=pd.read_csv("../merged/len3_ship.csv",encoding="shift-jis",index_col=0)
# whether count or not
count=0
# drop never treated group
data=data.dropna(subset=["release_year"])
data.head()

,year,id,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,stock_rx_lag,stock_otc_lag,stock_self_lag,Approved,KEGG,Ingredient,date,release_year,成分,variable
3,2008,114,解熱鎮痛消炎剤,113733431,32062003.0,0,16419098.0,1829448.0,0.0,NaN,NaN,NaN,1994.0,D00127,Piroxicam,1995/1/24,1995.0,ピロキシカム,id1
4,2008,114,解熱鎮痛消炎剤,113733431,32062003.0,0,16419098.0,1829448.0,0.0,NaN,NaN,NaN,1994.0,D00132,Ketoprofen,1995/2/10,1995.0,ケトプロフェン,id1
5,2008,114,解熱鎮痛消炎剤,113733431,32062003.0,0,16419098.0,1829448.0,0.0,NaN,NaN,NaN,2002.0,D01578,Pranoprofen,2004/2/16,2004.0,テルビナフィン塩酸塩,id1
6,2008,114,解熱鎮痛消炎剤,113733431,32062003.0,0,16419098.0,1829448.0,0.0,NaN,NaN,NaN,2009.0,D00904,Diclofenac sodium,2009/4/28,2009.0,ジクロフェナクナトリウム,id1
7,2008,114,解熱鎮痛消炎剤,113733431,32062003.0,0,16419098.0,1829448.0,0.0,NaN,NaN,NaN,2010.0,D01709,Loxoprofen sodium hydrate,2011/1/21,2011.0,ロキソプロフェンナトリウム水和物,id1


1 observation が複数行にまたがっているので経過年数ダミー以外は落として1行にまとめよう

In [3]:
data["elapsed"]=data["year"]-data["release_year"]
# data.loc[(data['elapsed'] > 10) | (data["elapsed"]< -5), 'elapsed'] = np.nan
# restrict year
data=data[data["release_year"]>2005]
data=data.drop_duplicates(subset=["id","year"])
dummies=pd.get_dummies(data["elapsed"],prefix="elapsed")
dummies.columns = dummies.columns.str.replace('-', 'm').str.replace(r'\..*', '', regex=True)
colnames=dummies.columns.to_list()
# count ver.
if count:
    aggregated_df = pd.concat([data.groupby(['year', 'id']).first(), dummies.groupby([data['year'], data['id']]).sum()], axis=1).reset_index()
else :
# dummy ver.
    aggregated_df = pd.concat([
        data.groupby(['year', 'id']).first(), 
        dummies.groupby([data['year'], data['id']]).apply(lambda x: (x[colnames] > 0).any().astype(int))
    ], axis=1).reset_index()
aggregated_df

,year,id,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,stock_rx_lag,...,elapsed_6,elapsed_7,elapsed_8,elapsed_9,elapsed_10,elapsed_11,elapsed_12,elapsed_13,elapsed_14,elapsed_15
0,2008,114,解熱鎮痛消炎剤,113733431,32062003.0,0,16419098.0,1829448.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
1,2008,123,自律神経剤,5180925,0.0,0,338320.0,0.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
2,2008,131,眼科用剤,200817924,43657879.0,0,27228595.0,6132288.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
3,2008,132,耳鼻科用剤,23901015,15040127.0,0,3550000.0,3634742.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,2008,218,高脂血症用剤,301791700,627095.0,0,33272955.0,66811.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,2021,339,その他の血液・体液用薬,160728799,0.0,0,43818347.0,0.0,0.0,48770190.0,...,0,0,1,0,0,0,0,0,0,0
290,2021,399,他に分類されない代謝性医薬品,865479417,7564617.0,0,265536926.0,1821.0,0.0,246043511.0,...,0,0,0,0,0,0,0,0,0,1
291,2021,449,その他のアレルギー用薬,226049764,7060159.0,6962008,51409796.0,4606525.0,4528553.0,60509613.0,...,0,0,0,0,0,0,0,0,0,1
292,2021,625,抗ウイルス剤,215855691,364231.0,364231,185416723.0,227307.0,227307.0,143728443.0,...,0,0,0,0,0,0,0,0,1,0


In [4]:
colnames

['elapsed_m7',
 'elapsed_m6',
 'elapsed_m5',
 'elapsed_m4',
 'elapsed_m3',
 'elapsed_m2',
 'elapsed_m1',
 'elapsed_0',
 'elapsed_1',
 'elapsed_2',
 'elapsed_3',
 'elapsed_4',
 'elapsed_5',
 'elapsed_6',
 'elapsed_7',
 'elapsed_8',
 'elapsed_9',
 'elapsed_10',
 'elapsed_11',
 'elapsed_12',
 'elapsed_13',
 'elapsed_14',
 'elapsed_15']

connect generic usage

In [5]:
dusage=pd.read_csv("../generic/generic_usage_imp.csv")
dusage
# generic from ndb
dgeneric=pd.read_csv("../merged/len3_ndb_generic_imputed.csv",index_col=0,encoding="shift-jis")
dgeneric


,薬効分類,year,generic_share_q,generic_share_r
0,114.0,2008.0,0.221254,0.052596
1,114.0,2009.0,0.221254,0.052596
2,114.0,2010.0,0.221254,0.052596
3,114.0,2011.0,0.221254,0.052596
4,114.0,2012.0,0.225977,0.054459
...,...,...,...,...
395,799.0,2019.0,0.006196,0.024777
396,799.0,2020.0,0.006056,0.022847
397,799.0,2021.0,0.007953,0.028184
398,799.0,2022.0,0.046758,0.042127


In [6]:
merged_df=aggregated_df.merge(dusage,left_on="year",right_on="Year")
merged_df=merged_df.merge(dgeneric,left_on=["id","year"],right_on=["薬効分類","year"])
merged_df

,year,id,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,stock_rx_lag,...,elapsed_11,elapsed_12,elapsed_13,elapsed_14,elapsed_15,Year,Percentage,薬効分類,generic_share_q,generic_share_r
0,2008,114,解熱鎮痛消炎剤,113733431,32062003.0,0,16419098.0,1829448.0,0.0,NaN,...,0,0,0,0,0,2008,35.35,114.0,0.221254,0.052596
1,2008,123,自律神経剤,5180925,0.0,0,338320.0,0.0,0.0,NaN,...,0,0,0,0,0,2008,35.35,123.0,0.192486,0.054872
2,2008,131,眼科用剤,200817924,43657879.0,0,27228595.0,6132288.0,0.0,NaN,...,0,0,0,0,0,2008,35.35,131.0,0.169267,0.065845
3,2008,132,耳鼻科用剤,23901015,15040127.0,0,3550000.0,3634742.0,0.0,NaN,...,0,0,0,0,0,2008,35.35,132.0,0.039823,0.056606
4,2008,218,高脂血症用剤,301791700,627095.0,0,33272955.0,66811.0,0.0,NaN,...,0,0,0,0,0,2008,35.35,218.0,0.242251,0.115100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2021,339,その他の血液・体液用薬,160728799,0.0,0,43818347.0,0.0,0.0,48770190.0,...,0,0,0,0,0,2021,79.00,339.0,0.813013,0.510858
276,2021,399,他に分類されない代謝性医薬品,865479417,7564617.0,0,265536926.0,1821.0,0.0,246043511.0,...,0,0,0,0,1,2021,79.00,399.0,0.437164,0.139218
277,2021,449,その他のアレルギー用薬,226049764,7060159.0,6962008,51409796.0,4606525.0,4528553.0,60509613.0,...,0,0,0,0,1,2021,79.00,449.0,0.608051,0.363450
278,2021,625,抗ウイルス剤,215855691,364231.0,364231,185416723.0,227307.0,227307.0,143728443.0,...,0,0,0,1,0,2021,79.00,625.0,0.502607,0.052158


In [7]:
id_dummies=pd.get_dummies(merged_df["id"],prefix="id")
df_with_dummies=pd.concat([merged_df,id_dummies],axis=1)
df_with_dummies.rename(columns={"Percentage":"generic_per"},inplace=True)
df_with_dummies

,year,id,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,stock_rx_lag,...,id_252,id_259,id_264,id_265,id_332,id_339,id_399,id_449,id_625,id_629
0,2008,114,解熱鎮痛消炎剤,113733431,32062003.0,0,16419098.0,1829448.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
1,2008,123,自律神経剤,5180925,0.0,0,338320.0,0.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
2,2008,131,眼科用剤,200817924,43657879.0,0,27228595.0,6132288.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
3,2008,132,耳鼻科用剤,23901015,15040127.0,0,3550000.0,3634742.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,2008,218,高脂血症用剤,301791700,627095.0,0,33272955.0,66811.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2021,339,その他の血液・体液用薬,160728799,0.0,0,43818347.0,0.0,0.0,48770190.0,...,0,0,0,0,0,1,0,0,0,0
276,2021,399,他に分類されない代謝性医薬品,865479417,7564617.0,0,265536926.0,1821.0,0.0,246043511.0,...,0,0,0,0,0,0,1,0,0,0
277,2021,449,その他のアレルギー用薬,226049764,7060159.0,6962008,51409796.0,4606525.0,4528553.0,60509613.0,...,0,0,0,0,0,0,0,1,0,0
278,2021,625,抗ウイルス剤,215855691,364231.0,364231,185416723.0,227307.0,227307.0,143728443.0,...,0,0,0,0,0,0,0,0,1,0


In [8]:
# df_with_dummies.to_csv("len3_agg.csv",encoding="shift-jis")
# dummy ver
if count==0:
    df_with_dummies.to_csv("../merged/len3_ship_cl_DN.csv",encoding="shift-jis")
else:
# count ver
    df_with_dummies.to_csv("../merged/len3_ship_sum_cl_DN.csv",encoding="shift-jis")